# Orders

✏️ **Exercise**

Today, we will investigate the **orders**, and their associated **review score**.

👉 Our goal is to create a DataFrame with the following features:


| feature_name 	| type 	| description 	|
|:---	|:---:	|:---	|
| `order_id` 	| str 	| the id of the order 	|
| `wait_time` 	| float 	| the number of days between order_date and delivered_date 	|
| `expected_wait_time` 	| float 	| the number of days between order_date and estimated_delivery_date 	|
| `delay_vs_expected` 	| float 	| if the actual delivery date is later than the estimated delivery date, returns the number of days between the two dates, otherwise return 0 	|
| `order_status` 	| str 	| the status of the order 	|
| `dim_is_five_star` 	| int 	| 1 if the order received a five-star review, 0 otherwise 	|
| `dim_is_one_star` 	| int 	| 1 if the order received a one_star, 0 otherwise 	|
| `review_score` 	| int 	| from 1 to 5 	|
| `number_of_products` 	| int 	| number of products that the order contains 	|
| `number_of_sellers` 	| int 	| number of sellers involved in the order 	|
| `price` 	| float 	| total price of the order paid by customer 	|
| `freight_value` 	| float 	| value of the freight paid by customer 	|
| `distance_customer_seller` 	| float 	| the distance in km between customer and seller (optional) 	|  
  
⚠️ We also want to filter out "non-delivered" orders, unless explicitly specified, otherwise we cannot compute the potential delays.

❓ **Your challenge**: 

- Implement each feature as a separate method within the `Order` class available at `olist/order.py`
- Then, create a method `get_training_data()` that returns the complete DataFrame **without `NaN`s**.

💡 Suggested methodology:
- Use the notebook below to write and test your code step-by-step first
- Then copy the code into `order.py` once you are certain of your code logic
- Focus on the data manipulation logic now, we will analyse the dataset visually in the next challenges

🔥 Notebook best practices (must-read) 👇

<details>
    <summary>▸ <i>click here</i></summary>

From now on, exploratory notebooks are going to become pretty long, and we strongly advise you to follow these notebook principles:
- Code your logic so that your Notebook can always be ran from top to bottom without crashing (Cell --> Run All)
- Name your variables carefully 
- Use dummy names such as `tmp` or `_` for intermediary steps when you know you won't need them for long
- Clear your code and merge cells when relevant to minimize Notebook size (`Shift-M`)
- Hide your cell output if you don't need to see it anymore (double-click on the red `Out[]:` section to the left of your cell).
- Make heavy use of jupyter nbextention `Collapsible Headings` and `Table of Content` (call a TA if you can't find them)
- Use the following shortcuts 
    - `a` to insert a cell above
    - `b` to insert a cell below
    - `dd` to delete a cell
    - `esc` and `arrows` to move between cells
    - `Shift-Enter` to execute cell and move focus to the next one
    - use `Shift + Tab` when you are between method brackets e.g. `groupby()` to get the docs! Repeat a few times to open it permanently

</details>





In [1]:
# Auto reload imported module every time a jupyter cell is executed (handy for olist.order.py updates)
%load_ext autoreload
%autoreload 2

In [2]:
# Import usual modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Import olist data
from olist.data import Olist
olist = Olist()
data = olist.get_data()

In [4]:
# What datasets do we have access to now ? 
data.keys() 

dict_keys(['sellers', 'product_category_name_translation', 'orders', 'order_items', 'customers', 'geolocation', 'order_payments', 'order_reviews', 'products'])

In [5]:
orders = data['orders'].copy() # good practice to be sure not to modify your `data` variable

assert(orders.shape == (99441, 8))

In [7]:
orders

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00


## 1. Code `order.py`

### a) `get_wait_time`
    ❓ Return a Dataframe with:
           order_id, wait_time, expected_wait_time, delay_vs_expected, order_status


🎁 We give you the pseudo-code below 👇 for this first method:

> 1. Inspect the `orders` dataframe
2. Filter the dataframe on `delivered orders`
3. Handle `datetime`
    - Take time to understand what python [`datetime`](https://docs.python.org/3/library/datetime.html) objects are
    - and convert dates from "string" type to "pandas.datetime' using [`pandas.to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html)
4. Compute `wait_time`
5. Compute `expected_wait_time`
6. Compute `delay_vs_expected`
7. Check the new dataframe 
8. Once you are satisfied with your code, you can carefully copy-paste it from the notebook to to `olist/order.py`

<details>
    <summary>💡Hint</summary>

For both `wait_time` and `delay_vs_expected`, you need to subtract the relevant dates/timestamps to get the time difference between the `pandas.datetime` objects. Then, you can either use [`datetime.timedelta()`](https://docs.python.org/3/library/datetime.html#timedelta-objects) or [`np.timedelta64()`](https://numpy.org/doc/stable/reference/arrays.datetime.html#datetime-and-timedelta-arithmetic) to find out how many days that subtraction represents!

</details>

In [11]:
orders['order_status'].value_counts()

delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: order_status, dtype: int64

In [12]:
orders.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [8]:
# Handle Datetime
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])

In [9]:
orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']

0        8 days 10:28:40
1       13 days 18:46:08
2        9 days 09:27:40
3       13 days 05:00:36
4        2 days 20:58:23
              ...       
99436    8 days 05:13:56
99437   22 days 04:38:58
99438   24 days 20:37:34
99439   17 days 02:04:27
99440    7 days 16:11:00
Length: 99441, dtype: timedelta64[ns]

In [10]:
# compute just the number of days in each time delta
import datetime

#one_day_delta = datetime.timedelta(day=1) # a "timedelta" object of 1 day
one_day_delta = np.timedelta64(24, 'h') # a "timedelta64" object of 1 day


In [11]:
# Assign compute delay vs expected
orders.loc[:, 'wait_time'] = \
    (orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']) / one_day_delta

orders.loc[:, 'expected_wait_time'] = \
    (orders['order_estimated_delivery_date'] - orders['order_purchase_timestamp']) / one_day_delta

In [12]:
orders.loc[:, 'delay_vs_expected'] = \
    (orders['order_delivered_customer_date'] - orders['order_estimated_delivery_date']) / one_day_delta

In [18]:
orders.shape

(99441, 11)

In [19]:
test = orders['delay_vs_expected'][orders['delay_vs_expected']>0]
test.shape

(7827,)

In [20]:
test2 = orders['delay_vs_expected'].apply(lambda x: x if x > 0 else 0)
test2.shape

(99441,)

In [21]:
orders.loc[orders['delay_vs_expected'] < 0, 'delay_vs_expected'] = 0

In [22]:
orders['delay_vs_expected'].describe()

count    96476.000000
mean         0.774961
std          4.753103
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        188.975081
Name: delay_vs_expected, dtype: float64

In [62]:
%%time
# orders['delay_vs_expected'] = orders['delay_vs_expected'][orders['delay_vs_expected']>0] = 0

# orders.loc[orders['delay_vs_expected'] < 0, 'delay_vs_expected'] = 0

# orders.loc[:,'delay_vs_expected'] = orders['delay_vs_expected'].apply(lambda x: x if x > 0 else 0)

CPU times: user 34.6 ms, sys: 1.82 ms, total: 36.4 ms
Wall time: 35.7 ms


In [24]:
orders['delay_vs_expected'].value_counts()

0.000000     88649
1.776979         3
2.660694         2
3.617396         2
6.734421         2
             ...  
0.587164         1
19.573426        1
3.762303         1
9.742025         1
8.656412         1
Name: delay_vs_expected, Length: 7755, dtype: int64

In [13]:
orders[['order_id', 'wait_time', 'expected_wait_time', 'delay_vs_expected', 'order_status']]

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,8.436574,15.544063,-7.107488,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,13.782037,19.137766,-5.355729,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,9.394213,26.639711,-17.245498,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,13.208750,26.188819,-12.980069,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,2.873877,12.112049,-9.238171,delivered
...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,8.218009,18.587442,-10.369433,delivered
99437,63943bddc261676b46f01ca7ac2f7bd8,22.193727,23.459051,-1.265324,delivered
99438,83c1379a015df1e13d02aae0204711ab,24.859421,30.384225,-5.524803,delivered
99439,11c177c8e97725db2631073c19f07b62,17.086424,37.105243,-20.018819,delivered


In [27]:
orders['delay_vs_expected'].shape

(99441,)

👀 Check the dataframe you've just created. <br/> 

💪 When your code works, commit it to `olist/order.py` <br/>

🧪 Now, test it by running the following cell 👇 

In [15]:
# Test your code here
from olist.order import Order
Order().get_wait_time()

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,8.436574,15.544063,0.0,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,13.782037,19.137766,0.0,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,9.394213,26.639711,0.0,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,13.208750,26.188819,0.0,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,2.873877,12.112049,0.0,delivered
...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,8.218009,18.587442,0.0,delivered
99437,63943bddc261676b46f01ca7ac2f7bd8,22.193727,23.459051,0.0,delivered
99438,83c1379a015df1e13d02aae0204711ab,24.859421,30.384225,0.0,delivered
99439,11c177c8e97725db2631073c19f07b62,17.086424,37.105243,0.0,delivered


In [16]:
from nbresult import ChallengeResult
result = ChallengeResult('wait_time', shape=Order().get_wait_time().shape)
result.write(); print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/bingobango/.pyenv/versions/tom/bin/python3
cachedir: .pytest_cache
rootdir: /Users/bingobango/code/lewagon/data-orders/tests
plugins: anyio-3.6.1, asyncio-0.19.0, typeguard-2.13.3
asyncio: mode=strict
collecting ... collected 1 item

test_wait_time.py::TestWaitTime::test_wait_time PASSED                   [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/wait_time.pickle

git commit -m 'Completed wait_time step'

git push origin master



### b) `get_review_score`
     ❓ Returns a DataFrame with:
        order_id, dim_is_five_star, dim_is_one_star, review_score

dim_is_$N$_star should contain `1` if review_score=$N$ and `0` otherwise 

<details>
    <summary markdown='span'>Hints</summary>

Think about `Series.map()` or `DataFrame.apply()`
    
</details>

👉 We load the `reviews` for you

In [30]:
reviews = data['order_reviews'].copy()
assert(reviews.shape == (99224,7))
reviews

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
...,...,...,...,...,...,...,...
99219,574ed12dd733e5fa530cfd4bbf39d7c9,2a8c23fee101d4d5662fa670396eb8da,5,NaN,NaN,2018-07-07 00:00:00,2018-07-14 17:18:30
99220,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5,NaN,NaN,2017-12-09 00:00:00,2017-12-11 20:06:42
99221,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43
99222,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4,NaN,NaN,2018-07-01 00:00:00,2018-07-02 12:59:13


In [31]:
dim_is_one_star = lambda x: int(x==1)
dim_is_five_star = lambda x: int(x==5)

In [35]:
# reviews['dim_is_five_star'] = reviews['review_score'].apply(dim_is_five_star)
# reviews['dim_is_one_star'] = reviews['review_score'].apply(dim_is_one_star)

In [38]:
reviews['dim_is_five_star'] = reviews['review_score'].apply(lambda x: int(x==5))
reviews['dim_is_one_star'] = reviews['review_score'].apply(lambda x: int(x==1))

In [39]:
reviews[["order_id", "dim_is_five_star", "dim_is_one_star", "review_score"]]

,order_id,dim_is_five_star,dim_is_one_star,review_score
0,73fc7af87114b39712e6da79b0a377eb,0,0,4
1,a548910a1c6147796b98fdf73dbeba33,1,0,5
2,f9e4b658b201a9f2ecdecbb34bed034b,1,0,5
3,658677c97b385a9be170737859d3511b,1,0,5
4,8e6bfb81e283fa7e4f11123a3fb894f1,1,0,5
...,...,...,...,...
99219,2a8c23fee101d4d5662fa670396eb8da,1,0,5
99220,22ec9f0669f784db00fa86d035cf8602,1,0,5
99221,55d4004744368f5571d1f590031933e4,1,0,5
99222,7725825d039fc1f0ceb7635e3f7d9206,0,0,4


Once again, 

👀 Check the dataframe you've just created. <br/> 

💪 When your code works, commit it to `olist/order.py` <br/>

🧪 Now, test it by running the following cell 👇 

In [17]:
# Test your code here
from olist.order import Order
Order().get_review_score()

,order_id,dim_is_five_star,dim_is_one_star,review_score
0,73fc7af87114b39712e6da79b0a377eb,0,0,4
1,a548910a1c6147796b98fdf73dbeba33,1,0,5
2,f9e4b658b201a9f2ecdecbb34bed034b,1,0,5
3,658677c97b385a9be170737859d3511b,1,0,5
4,8e6bfb81e283fa7e4f11123a3fb894f1,1,0,5
...,...,...,...,...
99219,2a8c23fee101d4d5662fa670396eb8da,1,0,5
99220,22ec9f0669f784db00fa86d035cf8602,1,0,5
99221,55d4004744368f5571d1f590031933e4,1,0,5
99222,7725825d039fc1f0ceb7635e3f7d9206,0,0,4


In [18]:
from nbresult import ChallengeResult
result = ChallengeResult('review_score', shape=Order().get_review_score().shape)
result.write(); print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/bingobango/.pyenv/versions/tom/bin/python3
cachedir: .pytest_cache
rootdir: /Users/bingobango/code/lewagon/data-orders/tests
plugins: anyio-3.6.1, asyncio-0.19.0, typeguard-2.13.3
asyncio: mode=strict
collecting ... collected 1 item

test_review_score.py::TestReviewScore::test_review_score PASSED          [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/review_score.pickle

git commit -m 'Completed review_score step'

git push origin master



### c) `get_number_products`:
     ❓ Returns a DataFrame with:
        order_id, number_of_products (total number of products per order)

In [20]:
order_items = data['order_items'].copy()
products = order_items.groupby("order_id", as_index=False).agg({'order_item_id': 'count'})
products.columns = ['order_id', 'number_of_products']

🧪 Same routine: 
* check your dataframe, 
* commit your code to `olist/order.py`
* and check that it truly works.

In [21]:
from nbresult import ChallengeResult
result = ChallengeResult('number_products', shape=Order().get_number_products().shape)
result.write(); print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/bingobango/.pyenv/versions/tom/bin/python3
cachedir: .pytest_cache
rootdir: /Users/bingobango/code/lewagon/data-orders/tests
plugins: anyio-3.6.1, asyncio-0.19.0, typeguard-2.13.3
asyncio: mode=strict
collecting ... collected 1 item

test_number_products.py::TestNumberProducts::test_review_score PASSED    [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/number_products.pickle

git commit -m 'Completed number_products step'

git push origin master



### d) `get_number_sellers`:
     ❓ Returns a DataFrame with:
        order_id, number_of_sellers (total number of unique sellers per order)
        
<details>
    <summary>▸ <i>Hint</i></summary>

`pd.Series.nunique()`
</details>

In [23]:
sellers = data['order_items'].copy()

sellers = sellers.groupby("order_id")['seller_id'].nunique().reset_index()
sellers.columns = ['order_id', 'number_of_sellers']
sellers.sort_values('number_of_sellers')
sellers

,order_id,number_of_sellers
0,00010242fe8c5a6d1ba2dd792cb16214,1
1,00018f77f2f0320c557190d7a144bdd3,1
2,000229ec398224ef6ca0657da4fc703e,1
3,00024acbcdf0a6daa1e931b038114c75,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,1
...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,1
98662,fffcd46ef2263f404302a634eb57f7eb,1
98663,fffce4705a9662cd70adb13d4a31832d,1
98664,fffe18544ffabc95dfada21779c9644f,1


In [24]:
from nbresult import ChallengeResult
result = ChallengeResult('number_sellers', shape=Order().get_number_sellers().shape)
result.write(); print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/bingobango/.pyenv/versions/tom/bin/python3
cachedir: .pytest_cache
rootdir: /Users/bingobango/code/lewagon/data-orders/tests
plugins: anyio-3.6.1, asyncio-0.19.0, typeguard-2.13.3
asyncio: mode=strict
collecting ... collected 1 item

test_number_sellers.py::TestNumberSellers::test_number_seller PASSED     [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/number_sellers.pickle

git commit -m 'Completed number_sellers step'

git push origin master



### e) `get_price_and_freight`
     Returns a DataFrame with:
        order_id, price, freight_value

<details>
    <summary>▸ <i>Hint</i></summary>

`pd.Series.agg()` allows you to apply one transformation method per column of your groupby object
</details>

In [25]:
data['order_items']

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [69]:
price_freight = data['order_items'].copy()

price_freight = price_freight.groupby('order_id', as_index=False).agg({'price': 'sum', 'freight_value': 'sum'})

price_freight[['order_id', 'price', 'freight_value']]

,order_id,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,199.90,18.14
...,...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,299.99,43.41
98662,fffcd46ef2263f404302a634eb57f7eb,350.00,36.53
98663,fffce4705a9662cd70adb13d4a31832d,99.90,16.95
98664,fffe18544ffabc95dfada21779c9644f,55.99,8.72


In [26]:
from nbresult import ChallengeResult
result = ChallengeResult('price', shape=Order().get_price_and_freight().shape)
result.write(); print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/bingobango/.pyenv/versions/tom/bin/python3
cachedir: .pytest_cache
rootdir: /Users/bingobango/code/lewagon/data-orders/tests
plugins: anyio-3.6.1, asyncio-0.19.0, typeguard-2.13.3
asyncio: mode=strict
collecting ... collected 1 item

test_price.py::TestPrice::test_price PASSED                              [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/price.pickle

git commit -m 'Completed price step'

git push origin master



### e) [OPTIONAL] `get_distance_seller_customer` 
**(Try  to code this function only after finishing today's challenges - Skip to next section)**

    ❓ Returns a Dataframe with:
        order_id, distance_seller_customer (the distance in km between customer and seller)

💡Have a look at the `haversine_distance` formula we coded for you in the `olist.utils` module

In [64]:
matching_geo = Order().get_distance_seller_customer()

In [32]:
orders = data['orders'].copy()
order_items = data['order_items'].copy()
sellers = data['sellers'].copy()
customers = data['customers'].copy()

In [33]:
geo = data['geolocation'].copy()
geo = geo.groupby('geolocation_zip_code_prefix',as_index=False).first()
geo

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1001,-23.549292,-46.633559,sao paulo,SP
1,1002,-23.548318,-46.635421,sao paulo,SP
2,1003,-23.549032,-46.635313,sao paulo,SP
3,1004,-23.550116,-46.635122,sao paulo,SP
4,1005,-23.549819,-46.635606,sao paulo,SP
...,...,...,...,...,...
19010,99960,-27.953797,-52.029641,charrua,RS
19011,99965,-28.173892,-52.038447,agua santa,RS
19012,99970,-28.345143,-51.876926,ciriaco,RS
19013,99980,-28.389218,-51.846012,david canabarro,RS


In [34]:
# merge geolocation for sellers
sellers_mask_columns = ['seller_id', 'seller_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']

In [48]:
sellers_geo = sellers.merge(
                            geo,
                            how='left',
                            left_on='seller_zip_code_prefix',
                            right_on='geolocation_zip_code_prefix')[sellers_mask_columns]

In [49]:
sellers_geo

,seller_id,seller_zip_code_prefix,geolocation_lat,geolocation_lng
0,3442f8959a84dea7ee197c632cb2df15,13023,-22.898536,-47.063125
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,-22.382941,-46.946641
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,-22.910641,-43.176510
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,-23.657250,-46.610759
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,-22.971648,-46.533618
...,...,...,...,...
3090,98dddbc4601dd4443ca174359b237166,87111,-23.456431,-51.866369
3091,f8201cab383e484733266d1906e2fdfa,88137,-27.623801,-48.674286
3092,74871d19219c7d518d0090283e03c137,4650,-23.659845,-46.677882
3093,e603cf3fec55f8697c9059638d6c8eb5,96080,-31.744231,-52.328761


In [37]:
# merge geolocation for customers
customers_mask_columns = ['customer_id', 'customer_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']

In [38]:
customers_geo = customers.merge(
                            geo,
                            how='left',
                            left_on='customer_zip_code_prefix',
                            right_on='geolocation_zip_code_prefix')[customers_mask_columns]

In [39]:
customers_geo

,customer_id,customer_zip_code_prefix,geolocation_lat,geolocation_lng
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,-20.509897,-47.397866
1,18955e83d337fd6b2def6b18a428ac77,9790,-23.726853,-46.545746
2,4e7b3e00288586ebd08712fdd0374a03,1151,-23.527788,-46.660310
3,b2b6027bc5c5109e529d4dc6358b12c3,8775,-23.496930,-46.185352
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056,-22.987222,-47.151073
...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,3937,-23.587901,-46.501830
99437,e7b71a9017aa05c9a7fd292d714858e8,6764,-23.612294,-46.765787
99438,5e28dfe12db7fb50a4b2f691faecea5e,60115,-3.744128,-38.510859
99439,56b18e2166679b8a959d72dd06da27f9,92120,-29.956391,-51.167614


In [40]:
# match customers with sellers in one table
customer_sellers = customers.merge(orders, on='customer_id')\
.merge(order_items, on='order_id')\
.merge(sellers, on='seller_id')\
[['order_id', 'customer_id','customer_zip_code_prefix', 'seller_id', 'seller_zip_code_prefix']]

In [41]:
customer_sellers

,order_id,customer_id,customer_zip_code_prefix,seller_id,seller_zip_code_prefix
0,00e7ee1b050b8499577073aeb2a297a1,06b8999e2fba1a1fbc88172c00ba8bc7,14409,7c67e1448b00f6e969d365cea6b010ab,8577
1,b2059ed67ce144a36e2aa97d2c9e9ad2,4e7b3e00288586ebd08712fdd0374a03,1151,7c67e1448b00f6e969d365cea6b010ab,8577
2,951670f92359f4fe4a63112aa7306eba,b2b6027bc5c5109e529d4dc6358b12c3,8775,7c67e1448b00f6e969d365cea6b010ab,8577
3,997fd4b37386d10b57d4ac8cdec011a6,4c06b42fbf7b97ab10779cda5549cd1c,65075,7c67e1448b00f6e969d365cea6b010ab,8577
4,b63fff1ef8b6201df68bb611ee41129e,8276de07ef25225d412b8462d73f8664,90010,7c67e1448b00f6e969d365cea6b010ab,8577
...,...,...,...,...,...
112645,d35703faec55cda0ceeb4f50a4d584ae,d21101a81657215bf9d86ccc3da340a1,18650,4ce6e5f6c52515177e18c1c9361d8677,9895
112646,b494f624c631105b24fe497277085909,eda10d7eb51a675d000956fa1aabc7dd,39920,9140ef75cc8211c4035935e80e567204,87015
112647,f73b31435ce6dec43df056154c39a1ce,f0f671d4034e98cdf20f0c452d6db02b,7231,da4d149c0ddbac90557103ac0a0ec356,9861
112648,d76fe905f4588ad69ab3b36527bb267d,b90b857a277b080d8078332b3636ab58,76400,377bc18334c78dd2f1535ef22a3ad225,83015


In [42]:
# add the geoloc
matching_geo = customer_sellers.merge(sellers_geo, on='seller_id')\
.merge(customers_geo, on='customer_id', suffixes=('_seller', '_customer'))

In [43]:
# drop na()
matching_geo = matching_geo.dropna()

In [50]:
matching_geo

,order_id,customer_id,customer_zip_code_prefix_seller,seller_id,seller_zip_code_prefix_x,seller_zip_code_prefix_y,seller_city,seller_state,geolocation_zip_code_prefix,geolocation_lat_seller,geolocation_lng_seller,geolocation_city,geolocation_state,customer_zip_code_prefix_customer,geolocation_lat_customer,geolocation_lng_customer,distance_seller_customer
0,00e7ee1b050b8499577073aeb2a297a1,06b8999e2fba1a1fbc88172c00ba8bc7,14409,7c67e1448b00f6e969d365cea6b010ab,8577,8577,itaquaquecetuba,SP,8577.0,-23.482623,-46.374490,itaquaquecetuba,SP,14409,-20.509897,-47.397866,252.932704
1,b2059ed67ce144a36e2aa97d2c9e9ad2,4e7b3e00288586ebd08712fdd0374a03,1151,7c67e1448b00f6e969d365cea6b010ab,8577,8577,itaquaquecetuba,SP,8577.0,-23.482623,-46.374490,itaquaquecetuba,SP,1151,-23.527788,-46.660310,31.969082
2,951670f92359f4fe4a63112aa7306eba,b2b6027bc5c5109e529d4dc6358b12c3,8775,7c67e1448b00f6e969d365cea6b010ab,8577,8577,itaquaquecetuba,SP,8577.0,-23.482623,-46.374490,itaquaquecetuba,SP,8775,-23.496930,-46.185352,21.059856
3,997fd4b37386d10b57d4ac8cdec011a6,4c06b42fbf7b97ab10779cda5549cd1c,65075,7c67e1448b00f6e969d365cea6b010ab,8577,8577,itaquaquecetuba,SP,8577.0,-23.482623,-46.374490,itaquaquecetuba,SP,65075,-2.508543,-44.303020,1650.473373
4,b63fff1ef8b6201df68bb611ee41129e,8276de07ef25225d412b8462d73f8664,90010,7c67e1448b00f6e969d365cea6b010ab,8577,8577,itaquaquecetuba,SP,8577.0,-23.482623,-46.374490,itaquaquecetuba,SP,90010,-30.032150,-51.232552,721.854304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,d35703faec55cda0ceeb4f50a4d584ae,d21101a81657215bf9d86ccc3da340a1,18650,4ce6e5f6c52515177e18c1c9361d8677,9895,9895,sao bernardo do campo,SP,9895.0,-23.707347,-46.566874,sao bernardo do campo,SP,18650,-22.727403,-48.600084,237.724037
112646,b494f624c631105b24fe497277085909,eda10d7eb51a675d000956fa1aabc7dd,39920,9140ef75cc8211c4035935e80e567204,87015,87015,maringa,PR,87015.0,-23.428806,-51.971831,maringa,PR,39920,-15.895836,-40.185044,1431.800665
112647,f73b31435ce6dec43df056154c39a1ce,f0f671d4034e98cdf20f0c452d6db02b,7231,da4d149c0ddbac90557103ac0a0ec356,9861,9861,sbc,SP,9861.0,-23.695889,-46.581744,sao bernardo do campo,SP,7231,-23.454964,-46.469216,22.277941
112648,d76fe905f4588ad69ab3b36527bb267d,b90b857a277b080d8078332b3636ab58,76400,377bc18334c78dd2f1535ef22a3ad225,83015,83015,sao jose dos pinhais,PR,83015.0,-25.562246,-49.175014,sao jose dos pinhais,PR,76400,-14.518523,-49.131917,802.460968


In [45]:
from olist.utils import haversine_distance

In [46]:
matching_geo.loc[:,'distance_seller_customer'] = \
            matching_geo.apply(lambda row: haversine_distance(row['geolocation_lat_seller'],
                                                              row['geolocation_lng_seller'],  
                                                              row['geolocation_lat_customer'],
                                                              row['geolocation_lng_customer']),
                                                               axis=1)

In [47]:
matching_geo

,order_id,customer_id,customer_zip_code_prefix_seller,seller_id,seller_zip_code_prefix_x,seller_zip_code_prefix_y,seller_city,seller_state,geolocation_zip_code_prefix,geolocation_lat_seller,geolocation_lng_seller,geolocation_city,geolocation_state,customer_zip_code_prefix_customer,geolocation_lat_customer,geolocation_lng_customer,distance_seller_customer
0,00e7ee1b050b8499577073aeb2a297a1,06b8999e2fba1a1fbc88172c00ba8bc7,14409,7c67e1448b00f6e969d365cea6b010ab,8577,8577,itaquaquecetuba,SP,8577.0,-23.482623,-46.374490,itaquaquecetuba,SP,14409,-20.509897,-47.397866,252.932704
1,b2059ed67ce144a36e2aa97d2c9e9ad2,4e7b3e00288586ebd08712fdd0374a03,1151,7c67e1448b00f6e969d365cea6b010ab,8577,8577,itaquaquecetuba,SP,8577.0,-23.482623,-46.374490,itaquaquecetuba,SP,1151,-23.527788,-46.660310,31.969082
2,951670f92359f4fe4a63112aa7306eba,b2b6027bc5c5109e529d4dc6358b12c3,8775,7c67e1448b00f6e969d365cea6b010ab,8577,8577,itaquaquecetuba,SP,8577.0,-23.482623,-46.374490,itaquaquecetuba,SP,8775,-23.496930,-46.185352,21.059856
3,997fd4b37386d10b57d4ac8cdec011a6,4c06b42fbf7b97ab10779cda5549cd1c,65075,7c67e1448b00f6e969d365cea6b010ab,8577,8577,itaquaquecetuba,SP,8577.0,-23.482623,-46.374490,itaquaquecetuba,SP,65075,-2.508543,-44.303020,1650.473373
4,b63fff1ef8b6201df68bb611ee41129e,8276de07ef25225d412b8462d73f8664,90010,7c67e1448b00f6e969d365cea6b010ab,8577,8577,itaquaquecetuba,SP,8577.0,-23.482623,-46.374490,itaquaquecetuba,SP,90010,-30.032150,-51.232552,721.854304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,d35703faec55cda0ceeb4f50a4d584ae,d21101a81657215bf9d86ccc3da340a1,18650,4ce6e5f6c52515177e18c1c9361d8677,9895,9895,sao bernardo do campo,SP,9895.0,-23.707347,-46.566874,sao bernardo do campo,SP,18650,-22.727403,-48.600084,237.724037
112646,b494f624c631105b24fe497277085909,eda10d7eb51a675d000956fa1aabc7dd,39920,9140ef75cc8211c4035935e80e567204,87015,87015,maringa,PR,87015.0,-23.428806,-51.971831,maringa,PR,39920,-15.895836,-40.185044,1431.800665
112647,f73b31435ce6dec43df056154c39a1ce,f0f671d4034e98cdf20f0c452d6db02b,7231,da4d149c0ddbac90557103ac0a0ec356,9861,9861,sbc,SP,9861.0,-23.695889,-46.581744,sao bernardo do campo,SP,7231,-23.454964,-46.469216,22.277941
112648,d76fe905f4588ad69ab3b36527bb267d,b90b857a277b080d8078332b3636ab58,76400,377bc18334c78dd2f1535ef22a3ad225,83015,83015,sao jose dos pinhais,PR,83015.0,-25.562246,-49.175014,sao jose dos pinhais,PR,76400,-14.518523,-49.131917,802.460968


In [51]:
# Since an order can have multiple sellers,
# return the average of the distance per order

order_distance = matching_geo.groupby('order_id', as_index=False).agg({'distance_seller_customer': 'mean'})

In [52]:
order_distance['distance_seller_customer'].mean()

519.160562449313

👀 Check your new dataframe and commit your code to olist/order.py when it works. 

In [53]:
Order().get_distance_seller_customer()

,order_id,distance_seller_customer
0,00010242fe8c5a6d1ba2dd792cb16214,301.005664
1,00018f77f2f0320c557190d7a144bdd3,589.274140
2,000229ec398224ef6ca0657da4fc703e,312.495046
3,00024acbcdf0a6daa1e931b038114c75,301.951753
4,00042b26cf59d7ce69dfabb4e55b4fd9,646.221788
...,...,...
98172,fffc94f6ce00a00581880bf54a75a037,2755.314485
98173,fffcd46ef2263f404302a634eb57f7eb,354.848852
98174,fffce4705a9662cd70adb13d4a31832d,338.827218
98175,fffe18544ffabc95dfada21779c9644f,72.870476


🧪  Test your code

In [54]:
from nbresult import ChallengeResult

result = ChallengeResult('distance',
    mean = Order().get_distance_seller_customer()['distance_seller_customer'].mean())
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/bingobango/.pyenv/versions/tom/bin/python3
cachedir: .pytest_cache
rootdir: /Users/bingobango/code/lewagon/data-orders/tests
plugins: anyio-3.6.1, asyncio-0.19.0, typeguard-2.13.3
asyncio: mode=strict
collecting ... collected 1 item

test_distance.py::TestDistance::test_distance PASSED                     [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/distance.pickle

git commit -m 'Completed distance step'

git push origin master



## 2. All at once: `get_training_data`

❓ Time to code `get_training_data` making use of your previous coded methods, to gather all order features in one table

In [55]:
Order().get_training_data()

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status,dim_is_five_star,dim_is_one_star,review_score,number_of_products,number_of_sellers,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,8.436574,15.544063,0.0,delivered,0,0,4,1,1,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,13.782037,19.137766,0.0,delivered,0,0,4,1,1,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,9.394213,26.639711,0.0,delivered,1,0,5,1,1,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,13.208750,26.188819,0.0,delivered,1,0,5,1,1,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,2.873877,12.112049,0.0,delivered,1,0,5,1,1,19.90,8.72
...,...,...,...,...,...,...,...,...,...,...,...,...
96356,9c5dedf39a927c1b2549525ed64a053c,8.218009,18.587442,0.0,delivered,1,0,5,1,1,72.00,13.08
96357,63943bddc261676b46f01ca7ac2f7bd8,22.193727,23.459051,0.0,delivered,0,0,4,1,1,174.90,20.10
96358,83c1379a015df1e13d02aae0204711ab,24.859421,30.384225,0.0,delivered,1,0,5,1,1,205.99,65.02
96359,11c177c8e97725db2631073c19f07b62,17.086424,37.105243,0.0,delivered,0,0,2,2,1,359.98,81.18


🧪  Test it below

In [56]:
from nbresult import ChallengeResult
from olist.order import Order
data = Order().get_training_data()

result = ChallengeResult('training',
    shape=data.shape,
    columns=sorted(list(data.columns))
)
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/bingobango/.pyenv/versions/tom/bin/python3
cachedir: .pytest_cache
rootdir: /Users/bingobango/code/lewagon/data-orders/tests
plugins: anyio-3.6.1, asyncio-0.19.0, typeguard-2.13.3
asyncio: mode=strict
collecting ... collected 2 items

test_training.py::TestTraining::test_training_data_columns PASSED        [ 50%]
test_training.py::TestTraining::test_training_data_shape PASSED          [100%]

============================== 2 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/training.pickle

git commit -m 'Completed training step'

git push origin master



🏁 Congratulations! 

💾 Commit and push your notebook before starting the next challenge.